# mini-projeto de analise de sentimentos
### dataset: https://www.kaggle.com/datasets/kavita5/twitter-dataset-avengersendgame

## Importando as ferramentas

In [ ]:
import pandas as pd
import seaborn as sns
import re

In [ ]:
# se vc n tem a nltk instalada
!pip install nltk
import nltk
nltk.download()

## olhando os dados e fazendo ajustes iniciais

In [ ]:
df = pd.read_csv('./tweets.csv')
df.head()

In [ ]:
df.info()

In [ ]:
ds = df.loc[:, ['text', 'created', 'screenName', 'id']]
ds.loc['created'] = pd.to_datetime(df['created'])

In [ ]:
ds['text'] = ds['text'].astype('string')

In [ ]:
ds.dropna(inplace=True)
print(f"número de linhas: {ds.shape[0]}")
ds.head()

## pegando uma amostra e iniciando a analise

In [ ]:
work = ds['text'].sample(n=1000)
work = work.str.lower()

In [ ]:
work.iloc[:5]

In [ ]:
# usando regex e funções do python para retirar/substituir padrões indesejados

# substituindo cacacteres desconhecidos por nada(apagando)
work = work.apply(lambda s: re.sub(r'[^\x00-\x7f]','', s))
# substituindo a string 'rt @' por nada(apagando)
work = work.apply(lambda s: s.replace('rt @', ''))
#removendo todos os caracteres que não sejam letras do alfabeto e os que formam links (padrão http)
work = work.apply(lambda s: re.sub(r"\s|[^a-zA-Z]|http\S+",' ', s))
# removendo os espaços desnecessários
work = work.apply(lambda s: ' '.join(s.split()))

work.iloc[:5]

## filtragem "profunda"

In [ ]:
# tirando termos redundantes (stopwords) + reduzindo os termos aos seus radicais (stemming)

stopwords = nltk.corpus.stopwords.words("english")
stemmer = nltk.stem.PorterStemmer()
tokenizer = nltk.tokenize.word_tokenize

def filtragem(string):
    tokens = tokenizer(string)
    newStr = [stemmer.stem(token) for token in tokens if (token not in stopwords)]
    return ' '.join(newStr)

work = work.apply(filtragem)

In [ ]:
# vendo os 20 termos mais falados
fd = nltk.probability.FreqDist()

def contabilidade(s):
    tokens = tokenizer(s)
    for string in tokens:
        fd[string] += 1

work.apply(contabilidade)
fd.most_common(20)

## plotagem

In [ ]:
# usando um algoritimo de sentimentos pronto
from nltk.sentiment import SentimentIntensityAnalyzer

score = []
sa =  SentimentIntensityAnalyzer()
work.apply(lambda s: score.append(sa.polarity_scores(s)))
score[:5]

In [ ]:
scores = pd.DataFrame(score)
scores = {'classes': ['neg', 'neu', 'pos'],
          'pontos': [scores['neg'].sum(), scores['neu'].sum(), scores['pos'].sum()]}
scores = pd.DataFrame(scores, columns = ['classes', 'pontos'])

In [ ]:
sns.barplot(data=scores, x="classes", y ="pontos")

## Conclusões: 
- gráfico e as 20 plavras mais frequentes.